In [1]:
import math
import time

def cache(func):
    saved = {}
    def wraper(*args):
        if (func, args) in saved:
            return saved[func, args]
        result = func(*args)
        saved[func, args] = result
        return result
    return wraper

@cache
def is_prime(n):
    if n <= 1:
        return False
    if n == 2 or n == 3:
        return True
    if n%2 == 0:
        return False
    max_div = int(math.sqrt(n))
    
    for x in xrange(3, max_div+2, 2):
        if n%x == 0:
            return False
    else:
        return True

def prime_seive(s):
    """returns all primes less than s"""
    seive = [1]*(s+1)
    seive[0] = seive[1] = 0
    current_prime = 2
    i = current_prime
    while current_prime < s:
        while i < s:
            i += current_prime
            try:
                seive[i] = 0
            except IndexError:
                pass
        current_prime += 1
        try:
            while seive[current_prime] == 0:
                current_prime += 1
        except IndexError:
            break
        i = current_prime
    primes = []
    for i in xrange(len(seive)):
        if seive[i] == 1:
            primes.append(i)
    return primes


#Problem 51
By replacing the 1st digit of the 2-digit number *3, it turns out that six of the nine possible values: 13, 23, 43, 53, 73, and 83, are all prime.

By replacing the 3rd and 4th digits of 56**3 with the same digit, this 5-digit number is the first example having seven primes among the ten generated numbers, yielding the family: 56003, 56113, 56333, 56443, 56663, 56773, and 56993. Consequently 56003, being the first member of this family, is the smallest prime with this property.

Find the smallest prime which, by replacing part of the number (not necessarily adjacent digits) with the same digit, is part of an eight prime value family.

In [18]:
import itertools
 

def family_search(primes, target_family_size):
    primes_set = set(primes)
    for prime in primes:
        prime_str = str(prime)
        for mask in set(prime_str):
            canidates = (int(prime_str.replace(mask, str(n))) for n in xrange(10))
            family = {n for n in canidates if n in primes_set and n >= prime}
            if len(family) == target_family_size:
                return family
        

def family_search_2(primes, target_family_size):
    primes_set = set(primes)
    for repeats in range(len(str(primes[-1]))-1):
        for prime in primes:
            c = Counter(str(prime))
            for repeated_number in (str(x) for x in xrange(10)):
                if c[repeated_number] == repeats:
                    family = []
                    for x in (str(prime).replace(repeated_number, str(i)) for i in xrange(10)):
                        if x.startswith('0'):
                            continue
                        if int(x) in primes_set:
                            family.append(x)
                    if len(family) == target_family_size:
                        return family

primes = prime_seive(1000000)
time_start = time.time()
print 'found!: ', sorted(family_search(primes, 6))
print time.time()-time_start
        

NameError: name 'prime_seive' is not defined

#Problem 52
It can be seen that the number, 125874, and its double, 251748, contain exactly the same digits, but in a different order.

Find the smallest positive integer, x, such that 2x, 3x, 4x, 5x, and 6x, contain the same digits.

In [32]:
import itertools
import time

def digit_check(a,b):
    return sorted(str(a)) == sorted(str(b))

time_start = time.time()
i = 1
while True:
    if all(itertools.imap(digit_check, (a*i for a in xrange(2, 7)), itertools.repeat(i))):
        break
    i += 1
print i
print time.time()-time_start

142857
1.10256814957


#Problem 53
There are exactly ten ways of selecting three from five, 12345:

123, 124, 125, 134, 135, 145, 234, 235, 245, and 345

In combinatorics, we use the notation, 5C3 = 10.

In general,

nCr =	n!/r!(n−r)!
,where r ≤ n, n! = n×(n−1)×...×3×2×1, and 0! = 1.
It is not until n = 23, that a value exceeds one-million: 23C10 = 1144066.

How many, not necessarily distinct, values of  nCr, for 1 ≤ n ≤ 100, are greater than one-million?

In [2]:
from math import factorial

def ncr(n, r):
    return factorial(n)/(factorial(r)*factorial(n-r))

time_start = time.time()
total = 0
for n in xrange(23, 101):
    for r in xrange(n+1):
        if ncr(n,r)> 1000000:
            total += 1
print time.time()-time_start
print total


0.0735878944397
4075


#Problem 53
find the winner of a bunch of poker hands

In [79]:
from functools import total_ordering
from collections import Counter
with open('p054_poker.txt', 'ru') as f:
    rounds = f.readlines()

def get_rank(h):
    HAND_RANKS = [
        'HC',
        '1P',
        '2P',
        '3K',
        'S',
        'F',
        'FH',
        '4K',
        'SF'
    ]
    return HAND_RANKS.index(h.upper())
        
@total_ordering
class Card(object):
    def __init__(self, s):
        try:
            self.value = int(s[0])
        except ValueError:
            face_values = {
                't': 10,
                'j': 11,
                'q': 12,
                'k': 13,
                'a': 14
                }
            self.value = face_values[s[0]]
        self.suit = s[1]
        
    def __eq__(self, other):
        return self.value == other
    def __lt__(self, other):
        return self.value < other

class Hand(object):
    def __init__(self, cards):
        self.cards = sorted([Card(c) for c in cards])
        self.rank = None
        self.rank_lead = None
        self.high_cards = []
        self.second_pair = None
        self.rank_hand()
    
    def rank_hand(self):
        flush = False
        straight = False
        if all(card.suit == self.cards[0].suit for card in self.cards):
            flush = True
        if self.cards == range(self.cards[0].value, self.cards[-1].value+1):
            straight = True
        if straight and flush:
            self.rank = get_rank('SF')
            self.rank_lead = self.cards[-1]
            return
        elif flush:
            self.rank = get_rank('f')
            self.rank_lead = self.cards[-1]
            return
        elif straight:
            self.rank = get_rank('s')
            self.rank_lead = self.cards[-1]
        else:
            self.populate_pairs()
    
    def populate_pairs(self):
        hand_set = {c.value for c in self.cards}
        if len(hand_set) == 5:
            self.rank = get_rank('HC')
            self.rank_lead = self.cards[-1]
            self.high_cards = sorted(self.cards[:-1], reverse=True)
            return
        counter = Counter(c.value for c in self.cards)
        for card, count in counter.items():
            if count == 4:
                self.rank = get_rank('4K')
                self.rank_lead = card
                self.high_cards = [c for c in self.cards if c != card]
                return
            if count == 2 and len(hand_set) == 4:
                self.rank = get_rank('1P')
                self.rank_lead = card
                self.high_cards = sorted([c for c in self.cards if c != card], reverse=True)
                return
            if count == 3 and len(hand_set) == 3:
                self.rank = get_rank('3k')
                self.rank_lead = card
                self.high_cards = sorted([c for c in self.cards if c != card], reverse=True)
                return
            if count == 3 and len(hand_set) == 2:
                self.rank = get_rank("FH")
                self.rank_lead = card
                self.high_cards = [c for c in self.cards if c != card]
                return
        #must be 2p
        self.rank = get_rank('2p')
        self.high_cards = [card for card in counter if counter[card] == 1]
        self.rank_lead = [card for card in self.cards if card != self.high_cards[0]][-1]
        self.second_pair = [card for card in self.cards if card != self.high_cards[0]][0]
        return

def winner(h1, h2):
    if h1.rank > h2.rank:
        return h1
    if h1.rank < h2.rank:
        return h2
    # tie
    if h1.rank == get_rank('2p'):
        #special handeling of 2 pair
        if h1.rank_lead == h2.rank_lead:
            if h1.second_pair == h2.second_pair:
                print h1.high_cards
                return max([h1,h2], key=lambda hand: hand.high_cards[0])
            else:
                return max([h1,h2], key=lambda hand: hand.second_pair)
        else:
            return max([h1,h2], key=lambda hand: hand.rank_lead)

    if h1.rank_lead == h2.rank_lead:
        for hc1, hc2 in zip(h1.high_cards, h2.high_cards):
            if hc1 == hc2:
                continue
            else:
                if hc1 > hc2:
                    return h1
                else:
                    return h2

    return max([h1,h2], key=lambda hand: hand.rank_lead)

p1_wins = 0
for round in rounds:
    p1 = Hand(round.strip().lower().split()[:5])
    p2 = Hand(round.strip().lower().split()[5:])
    
    if winner(p1, p2) is p1:
        p1_wins +=1
print p1_wins

376


#Problem 55
If we take 47, reverse and add, 47 + 74 = 121, which is palindromic.

Not all numbers produce palindromes so quickly. For example,

349 + 943 = 1292,  
1292 + 2921 = 4213  
4213 + 3124 = 7337  

That is, 349 took three iterations to arrive at a palindrome.

Although no one has proved it yet, it is thought that some numbers, like 196, never produce a palindrome. A number that never forms a palindrome through the reverse and add process is called a Lychrel number. Due to the theoretical nature of these numbers, and for the purpose of this problem, we shall assume that a number is Lychrel until proven otherwise. In addition you are given that for every number below ten-thousand, it will either (i) become a palindrome in less than fifty iterations, or, (ii) no one, with all the computing power that exists, has managed so far to map it to a palindrome. In fact, 10677 is the first number to be shown to require over fifty iterations before producing a palindrome: 4668731596684224866951378664 (53 iterations, 28-digits).

Surprisingly, there are palindromic numbers that are themselves Lychrel numbers; the first example is 4994.

How many Lychrel numbers are there below ten-thousand?

NOTE: Wording was modified slightly on 24 April 2007 to emphasise the theoretical nature of Lychrel numbers.

In [111]:
def lychrel_check(n):
    itterations = 0
    while itterations < 50:
        itterations += 1
        reversed_n = int(str(n)[::-1]) #extended slice syntax for step size
        n += reversed_n
        str_n = str(n)
        if str_n == str_n[::-1]:
            return False
    return True
            
total = 0
s_time = time.time()
for x in xrange(10000):
    if lychrel_check(x):
        total += 1
print time.time()-s_time
print total

0.178150177002
249


#Problem 56
A googol (10^100) is a massive number: one followed by one-hundred zeros; 100^100 is almost unimaginably large: one followed by two-hundred zeros. Despite their size, the sum of the digits in each number is only 1.

Considering natural numbers of the form, a^b, where a, b < 100, what is the maximum digital sum?

In [116]:
def sum_digits(n):
    r = 0
    while n:
        r, n = r + n % 10, n / 10
    return r
s_time = time.time()
maximum_digital_sum = 0
for a in xrange(90, 100):
    for b in xrange(90, 100):
        digital_sum = sum_digits(a**b)
        if digital_sum > maximum_digital_sum:
            maximum_digital_sum = digital_sum
print time.time() - s_time
print maximum_digital_sum


0.0232720375061
972


#Problem 57
It is possible to show that the square root of two can be expressed as an infinite continued fraction.

√ 2 = 1 + 1/(2 + 1/(2 + 1/(2 + ... ))) = 1.414213...  

By expanding this for the first four iterations, we get:

1 + 1/2 = 3/2 = 1.5  
1 + 1/(2 + 1/2) = 7/5 = 1.4  
1 + 1/(2 + 1/(2 + 1/2)) = 17/12 = 1.41666...  
1 + 1/(2 + 1/(2 + 1/(2 + 1/2))) = 41/29 = 1.41379...  

The next three expansions are 99/70, 239/169, and 577/408, but the eighth expansion, 1393/985, is the first example where the number of digits in the numerator exceeds the number of digits in the denominator.

In the first one-thousand expansions, how many fractions contain a numerator with more digits than denominator?

In [134]:
from fractions import Fraction
total = 0
a = Fraction(1,2)
for _ in xrange(1000):
    a = Fraction(1, a+2)
    if len(str((1+a).numerator)) > len(str((1+a).denominator)):
        total +=1 
print total

153


#Problem 58
Starting with 1 and spiralling anticlockwise in the following way, a square spiral with side length 7 is formed.

37 36 35 34 33 32 31  
38 17 16 15 14 13 30  
39 18  5  4  3 12 29  
40 19  6  1  2 11 28  
41 20  7  8  9 10 27  
42 21 22 23 24 25 26  
43 44 45 46 47 48 49  

It is interesting to note that the odd squares lie along the bottom right diagonal, but what is more interesting is that 8 out of the 13 numbers lying along both diagonals are prime; that is, a ratio of 8/13 ≈ 62%.

If one complete new layer is wrapped around the spiral above, a square spiral with side length 9 will be formed. If this process is continued, what is the side length of the square spiral for which the ratio of primes along both diagonals first falls below 10%?

In [38]:
def sum_diag():
    diagonals = []
    primes = 0
    non_primes = 0
    totals = 1
    x = 1
    while True:
        layers += 1
        for _ in range(4):
            totals += (x*2)
            diagonals.append(totals)
        for n in diagonals[-4:]:
            if is_prime(n):
                primes += 1
            else:
                non_primes += 1
        if float(primes) / ((primes +non_primes)+1) < 0.1:
            break
        x += 1
    print 2*x+1, float(primes)/(len(diagonals)+1)

sum_diag()

26241 0.0999980945485


#Problem 60
The primes 3, 7, 109, and 673, are quite remarkable. By taking any two primes and concatenating them in any order the result will always be prime. For example, taking 7 and 109, both 7109 and 1097 are prime. The sum of these four primes, 792, represents the lowest sum for a set of four primes with this property.

Find the lowest sum for a set of five primes for which any two primes concatenate to produce another prime.

In [3]:
import itertools
primes = prime_seive(10000)

def test_permutations(lst):
    for p in itertools.permutations(lst,2):
        if not is_prime(int(''.join(str(x) for x in p))):
            return False
    else:
        return True
@cache
def compute_set(p):
    return {prime for prime in primes if test_permutations((prime,p))}

iterations = 0
target_depth = 5
def plumb_chain(chain):
    if len(chain) == target_depth:
        return chain
    intersection = set.intersection(*[compute_set(s) for s in chain])
    if len(intersection) == 0:
        #deepest we can go
        return False
    for prime in intersection:
        if prime < chain[-1]:
            continue
        new_chain = plumb_chain(chain + [prime])
        if new_chain:
            return new_chain
s_time = time.time()
for p in primes:
    chain = plumb_chain([p])
    if chain:
        print 'CHAIN: ', chain
        break
print time.time()-s_time
print 'ANSWER: ',sum(chain)

CHAIN:  [13, 5197, 5701, 6733, 8389]
9.22900009155
ANSWER:  26033


#Problem 61
Triangle, square, pentagonal, hexagonal, heptagonal, and octagonal numbers are all figurate (polygonal) numbers and are generated by the following formulae:

Triangle	 	P3,n=n(n+1)/2	 	1, 3, 6, 10, 15, ...  
Square	 	P4,n=n2	 	1, 4, 9, 16, 25, ...  
Pentagonal	 	P5,n=n(3n−1)/2	 	1, 5, 12, 22, 35, ...  
Hexagonal	 	P6,n=n(2n−1)	 	1, 6, 15, 28, 45, ...  
Heptagonal	 	P7,n=n(5n−3)/2	 	1, 7, 18, 34, 55, ...  
Octagonal	 	P8,n=n(3n−2)	 	1, 8, 21, 40, 65, ...  
The ordered set of three 4-digit numbers: 8128, 2882, 8281, has three interesting properties.

- The set is cyclic, in that the last two digits of each number is the first two digits of the next number (including the last number with the first).
- Each polygonal type: triangle (P3,127=8128), square (P4,91=8281), and pentagonal (P5,44=2882), is represented by a different number in the set.
- This is the only set of 4-digit numbers with this property.

Find the sum of the only ordered set of six cyclic 4-digit numbers for which each polygonal type: triangle, square, pentagonal, hexagonal, heptagonal, and octagonal, is represented by a different number in the set.

In [3]:
import itertools
from math import factorial

setup_time = time.time()
geo_func = {
    'tri': lambda n: n*(n+1)/2,
    'sqr': lambda n: n*n,
    'pent': lambda n: n*(3*n-1)/2,
    'hex': lambda n: n*(2*n-1),
    'hept': lambda n: n*(5*n-3)/2,
    'oct': lambda n: n*(3*n-2)
}

def gen_geometric(operator):
    n = 0
    hex = []
    while True:
        n += 1
        num = operator(n)
        if num < 1000:
            continue
        if num > 10000:
            break
        hex.append(num)
    return hex
series_dict = {}
for key, opt in geo_func.items():
    series_dict[key] = gen_geometric(opt)
series = list(set(itertools.chain.from_iterable(series_dict.values()))) + gen_geometric(geo_func['hex'])
series = sorted([x for x in series if str(x)[-2] is not '0'])
unique_endings = {str(s)[-2:] for s in series}
unique_beginings = {str(s)[:-2] for s in series}
requirements = set.intersection(unique_beginings, unique_endings)
series = [x for x in series if all([str(x)[:-2] in requirements, str(x)[-2:] in requirements])]

for key, numbers in series_dict.items():
    series_dict[key] = {str(v) for v in numbers if v in series}
print "SETUP TIME: ", time.time()-setup_time
    
def follow_chain(chain, order):
    if len(chain) == 5:
        final = chain[-1][2:] + chain[0][:2]
        if final in series_dict[order[0]]:
            print "THREAD PULLED", chain + [final]
            return chain + [final]
        return False
    else:
        next_pair = chain[-1][2:]
        for n in series_dict[order[0]]:
            if n.startswith(next_pair):
                return follow_chain(chain+ [n], order[1:])
    return False

time_n = time.time()
for p in itertools.permutations(series_dict.keys(), 6):
    for n in series_dict[p[0]]:
        chain = [n]
        new_chain = follow_chain(chain, p[1:])
        if new_chain:
            print "BREAKING"
            break
    if new_chain:
        break

print time.time()-time_n
        
@cache
def get_matching_numbers(series_key, number):
        return {n for n in series_dict[series_key] if n.startswith(number[2:])}

def get_chain(series_dict, chain, remaining_series):
    if len(chain) == 5:
        if (chain[-1][2:] + chain[0][:2]) in series_dict[remaining_series[0]]:
            print "BOOM!"
            chain.append(chain[-1][2:]+chain[0][:2])
            return chain
        else:
            return False
    for s in remaining_series:
        for n in get_matching_numbers(s, chain[-1]):
            new_chain = get_chain(series_dict, chain+[n], [r for r in remaining_series if r != s])
            if new_chain:
                return new_chain
    return False


print sum((int(n) for n in new_chain))

time_o = time.time()
for n in series_dict['pent']:
    chain = get_chain(series_dict, [n], [key for key in series_dict if key != 'pent'])
    if chain:
        print "DONE:", chain
        break
print time.time()-time_o
print sum((int(n) for n in chain))
#sort back into dict
#chain walk through each list?

SETUP TIME:  0.00200009346008
THREAD PULLED ['8256', '5625', '2512', '1281', '8128', '2882']
BREAKING
0.039999961853
28684
BOOM!
DONE: ['2882', '8256', '5625', '2512', '1281', '8128']
0.0019998550415
28684


DONE: ['3321', '2116', '1681', '8128', '2821', '2133']
    